## Getting links by sending seperate get requests for each link - faster

In [22]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

# Setting up Web Driver
options = Options()
#options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)
driver.get('https://guide.wisc.edu/courses/')

# Initializing empty dict for class dict and list for class information (requisites, repeatable, etc.)
class_list = []
detail_keys = ['requisites', 'designation', 'repeatable', 'last_taught']

majors = driver.find_element(By.ID, "atozindex")
links = majors.find_elements(By.TAG_NAME, "a")

# Gather all the URLs first
urls = [link.get_attribute('href') for link in links]

# Now visit each URL directly
for i in range(len(urls))[:5]:
    driver.get(urls[i])

    classes = WebDriverWait(driver, 20, ignored_exceptions=(NoSuchElementException, StaleElementReferenceException)).until(EC.presence_of_element_located((By.CLASS_NAME,  "sc_sccoursedescs")))
    titles = classes.find_elements(By.CLASS_NAME, "courseblockcode")
    credits = classes.find_elements(By.CLASS_NAME, "courseblockcredits")
    buttons = classes.find_elements(By.XPATH,  "//*[text()='View details']")
    for i in range(len(buttons)):
        class_dict = {}
        class_dict['class'] = titles[i].text
        class_dict['credits'] = int(credits[i].text[:1])
        class_list.append(class_dict)

        buttons[i].click()
        time.sleep(0.5)

        # details = classes.find_elements(By.CLASS_NAME, "cbextra-data")
        # class_details = []
        # for detail in details:
        #     class_details.append(detail.text)
        #     temp_dict = {}
        #     for k, v in zip(detail_keys, class_details):
        #         temp_dict[k] = v
            #class_dict['class info'] = temp_dict
        #class_list.append(class_dict)
driver.quit()
print(class_list)
